# Introduction
This is hydraulic fracturing data analysis, for data gathered from 175 wells in the western desert in Egypt! The purpose of this analysis is to get some insights about fracturing operations in Egypt.

**Questions to answer from this data analysis:**
1. What is the most used polymer type and size?
2. Is there a relationship between polymer type/size and pumping rate with the formation?
3. The closure pressure, ISIP and max treating pressure for each foramtion
4. Indication of proppant volume and slurry volume for formations.
5. What is the relationship between max treating pressure, hhp, pumping rate and different formations?
6. What is the relationship between fluid efficiency and other variables and its effect on max treating pressure?
7. Does propant type/size affect closure pressure? And what role does frac type have?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df = pd.read_csv('clean_data_October_24_original.csv')
df.head()

In [ ]:
df.columns

> First step would be to investigate the size of our dataset, some statistics the availability of data in each feature and to quickly find out which features lack data, and the correlation between different features.

# 1.  Data Cleaning

### A) A quick look over the dataset

In [ ]:
# Getting the total number of null values in the dataset and then extract the non-null values
available_data = (len(df) - pd.isnull(df).sum().sort_values(ascending = False))
# Plotting features vs. available data on a bar plot
plt.figure(figsize = (12,8))
available_data.plot(kind = 'barh')
#plt.xticks(np.arange(0,1.1,0.1))
plt.xlabel('Non-Null count')
plt.ylabel('Features')
plt.title('Non-Null data valeus for each feature for 175 data points');

**We can here find ou that out of 175 data points and 20 features, we can see that:** 
* 14 features have more than 70% (or 125 data points) of the dataset with non null data that we could readily use.
* 3 features have around 30% (or 50 data points) of the dataset with non null values.
* 2 features have around 20% (or 30 data points) of the dataset with non null values.
* 1 feature has only one data point, which we should ignore right now till we gather further data.

In [ ]:
# Shape of our dataset
print('There are %d data observations and %d features in our dataset!'%(df.shape[0], df.shape[1]))

### B) quick statistical analysis

In [ ]:
# statistical data for numeric values
numerical_describe = df.describe()
numerical_describe

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df[df.success == 1].describe()

In [ ]:
df[df.success == 0].describe()

In [ ]:
# statistical data for categorical values
categorical_describe = df.describe(include = [object])
categorical_describe.loc['percentage',:] = categorical_describe.loc['freq', :]/categorical_describe.loc['count', :]
categorical_describe

In [ ]:
df[df.success == 0]['formation'].value_counts()

In [ ]:
df[df.success == 1]['formation'].value_counts()

In [ ]:
df[df.success == 1][categorical_features].value_counts()

**From quick statisitcal analysis, we can find out the following points:**
1. Most frac jobs are performed for perforation intervals with average of 28 ft, yet some intervals exceeded 100 ft so we need to check the performance of frac job in such exceptional jobs!
2. Various frac jobs had wide standard deviation of the pad volume pumped that varies up to 270,000 gal! We might be interested to investigate the reason for this large variety and whether it is related to job size or job conditions!
3. Again, we can see that 75% of proppant mass pumped did not exceed 100,000 lbs yet sime of them reached 600,000 lb and on the other hand, some operations used only 1000 lbs of proppant which need frther investigation.
4. Closure pressures range between 2000 to 12,000 psig, and similarly is max treating pressure and less likely is ISIP !
5. Fluid efficiency ranged from 22% to 78%, HHP ranged from 500 to 15,000 hp while slurry average pumping rate was 28 bpm.
6. 85% (or 142 out of 166) of the avaialble frac jobs are successful jobs.
7. There are 11 types of proppant used yet Ceramics is the most used proppant type and it accounts for 51% (or 64 out of 124) of all proppants used.
8. There are 10 mesh sizes used while some jobs included various mesh sizes where the most used proppant size is 16/13 and it represents 57% (or 71 out of 124) mesh sizes use.
9. All jobs are carried out by two different service companies, where Schlumberger accounted for 68% of the jobs (or 120 out of 175 jobs).
10. Frac jobs are mainly used along 6 different formations while ARG is the most fractured zone as it accoutns for 43% (or 71/163) of total jobs.

### C) correlation Matrix

In [ ]:
# Assigning numerical and categorical features to use later
numerical_features = ['pad_volume_gal', 'slurry_vol_gal', 'pad_perc_as_fraction', 'propp_mass_lb', 'propp_mass_design_lb', 
                      'max_surface_prop_conc_ppa', 'closure_pressure_psig',  'net_pressure_psig', 'BHISIP_psig', 
                      'max_treating_pressure_psig', 'perf_friction_psig', 'near_wellbore_friction', 'avg_hhp', 'slurry_rate_bpm',
                      'frac_width_ft', 'frac_half_length_ft', 'frac_height_ft', 'avg_conductivity_md_ft', 
                      'dimensionless_effectived_fcd', 'gel_retained_factor', 'gel_concentration_lb_mgal']
categorical_features = ['proppant_type', 'service_company', 'proppant_size', 'main_formation', 'frac_type']

In [ ]:
# Correlation matrix for categorical data
# Cramer's V method is used here to find correlation between categorical data
# https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9
def cramers_v(x, y):
    import scipy.stats as ss
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

def corr_cat(df, data):
    import itertools
    # COlumns to generate correlation matrix
    cols = data
    # Making zeros matrix with the length of items
    corrM = np.zeros((len(cols),len(cols)))
    # iterations through all possible two combinations of the list
    for col1, col2 in itertools.combinations(cols, 2):
        # Setting the index name
        idx1, idx2 = cols.index(col1), cols.index(col2)
        # Calculating the corr using the defined function
        corrM[idx1, idx2] = cramers_v(df[col1], df[col2])
        # To have all values witth same number of decimals
        corrM[idx2, idx1] = corrM[idx1, idx2]

    # Now we have our matrix
    corr = (pd.DataFrame(corrM, index=cols, columns=cols)).T
    return(corr)

In [ ]:
# Creating Correlation matrix for numerical features
numerical_correlation_matrix = df[numerical_features].corr()
numerical_mask = np.zeros_like(numerical_correlation_matrix)
numerical_mask[np.triu_indices_from(numerical_mask)] = True
# Plotting correlation matrix as a triu
plt.figure(figsize = (20,8))
sns.heatmap(numerical_correlation_matrix#, mask = numerical_mask
            , annot = True, cmap = 'coolwarm');
plt.title('Correlation Matrix among numerical features for Frac job');

In [ ]:
# Creating Correlation matrix for numerical features
categorical_correlation_matrix = corr_cat(df, categorical_features)
categorical_mask = np.zeros_like(categorical_correlation_matrix)
categorical_mask[np.triu_indices_from(categorical_mask)] = True
# Plotting correlation matrix as a triu
plt.figure(figsize = (12,8))
sns.heatmap(categorical_correlation_matrix, mask = categorical_mask, annot = True, cmap = 'coolwarm');
plt.title('Correlation Matrix among categorical features for Frac job');

**From the correlation matrix, we can see some insights from the available data such that:** 
* Success: it is kind of negatively related to pressure readings of (closure - ISIP - max treating) and HHP.
* HHP: strongly correlated to pressure readings, pumping rate and pad voume design.
* Fluid efficiency: It is almost not correlated to other features yet weakly negatively correlated to proppant mass pumped!
* Proppant mass design: weak correlation to other features, and strongly correlated to the actual pumped proppant"logically"!
* Proppant mass pumped: very strongly correlated to slurry pumped volume "logically"!
* Pad volume: intermediately positively correlated with HHP, max treating pressure and slurry volume "Surprisngly not a strong correlation", yet there is a slight negative trned with success of the operation.
* Slurry volume: an intermediate positive rleationship with slurry pumping rate, which makes sense since it is required to finish the operation ASAP.
* Closure pressure / Max treating pressure: There is a srong correlation with HHP, and max treating pressure yet a strong negative correlation with the success of the operation.
* Service company: we can see it is strongly affects the proppant type "logical"! and it less likely determines the proppant size. Yet, we can see that service companies operations kind of dependent on the foramtion type! 

**Correlation Matrix Notes:**
1. Throughout 120 observations, proppant mass pumped and (prop mass design - max proppant concentration PPA) are strongly correalted. As well, Throughout 49 observations, we can see that slurry volume and prop mass pumped are strongly correlated
2. Throughout 27 observations, we can see that Max treating pressure and all (BHISIP - closure pressure) vales are strongly correlated.
3. Service company is strongly correlated to the commercial name of the proppant type.

**Conclusion:**
* we will drop closure pressure, BHSIP, slurry volume, ppa, prop design, service company name

**Combinations:**
1. we will combine near wellbore friction with perforation friction as a new total friction value.\
2. we will drop reservoir pressure, frac gradient and closure pressure gradient for rare available data

In [ ]:
df.drop(['pad_volume_gal', 'service_company', 'formation', 'propp_mass_design_lb', 'max_surface_prop_conc_ppa',
         'slurry_vol_gal', 'BHISIP_psig', 'closure_pressure_psig', 'closure_gradient_psi_ft',
         'frac_gradient_psi_ft', 'reservoir_pressure_psig', 'surface_isip_psig'], axis =1, inplace = True)

In [ ]:
df['year'] = pd.DatetimeIndex(df['date']).year
bins = [2007, 2013, 2016, 2020]
names = ['08 to 13', '14 to 16', '17 to 20']
df['year'] = pd.cut(df['year'], bins, labels=names)

# Clean Dataset

In [ ]:
df = df[df['net_pressure_psig'].notna()]

In [ ]:
df = df[df['proppant_type'].notna()]

In [ ]:
df.head()

In [ ]:
# Getting the total number of null values in the dataset and then extract the non-null values
available_data = (len(df) - pd.isnull(df).sum().sort_values(ascending = False))
# Plotting features vs. available data on a bar plot
plt.figure(figsize = (12,8))
available_data.plot(kind = 'barh')
#plt.xticks(np.arange(0,1.1,0.1))
plt.xlabel('Non-Null count')
plt.ylabel('Features')
plt.title('Non-Null data valeus for each feature for 175 data points');

In [ ]:
pairplot_data = df[['success', 'propp_mass_lb', 'net_pressure_psig', 'slurry_rate_bpm',
                    'max_treating_pressure_psig', 'avg_hhp', 'fluid_eff_frac', 'frac_height_ft',
                    'frac_half_length_ft', 'avg_conductivity_md_ft', 'gel_retained_factor', 'perf_friction_psig', 'near_wellbore_friction',
                    'frac_type', 'proppant_type', 'proppant_size', 'main_formation']]

In [ ]:
sns.pairplot(pairplot_data, hue = 'success');

In [ ]:
sns.pairplot(pairplot_data, hue = 'service_company');

In [ ]:
sns.pairplot(pairplot_data, hue = 'proppant_size');

In [ ]:
sns.pairplot(pairplot_data, hue = 'main_formation');

In [ ]:
pairplot_data = df[['propp_mass_lb', 'net_pressure_psig', 'slurry_rate_bpm',
                    'max_treating_pressure_psig', 'avg_hhp', 'fluid_eff_frac', 'frac_height_ft',
                    'frac_half_length_ft', 'avg_conductivity_md_ft', 'gel_retained_factor', 'perf_friction_psig', 'near_wellbore_friction',
                    'success', 'frac_type', 'proppant_type', 'proppant_size', 'main_formation', 'service_company']]
sns.pairplot(pairplot_data, hue = 'success');

# Insights

In [ ]:
# Histogram fo various features
df.hist(figsize = (20,20), layout = (6,5));
plt.title('Histogram of different features');

In [ ]:
numerical_data = ['perf_interval_ft','propp_mass_lb', 'max_treating_pressure_psig','perf_friction_psig', 'slurry_rate_bpm',
                  'fluid_eff_frac', 'avg_hhp', 'near_wellbore_friction', 'frac_half_length_ft', 'frac_height_ft', 
                  'frac_width_ft', 'pad_perc_as_fraction', 'avg_conductivity_md_ft', 'dimensionless_effectived_fcd',
                  'net_pressure_psig', 'gel_retained_factor', 'gel_concentration_lb_mgal', 'propped_width_in']

categorical_data = ['success', 'proppant_type', 'proppant_size', 'frac_type', 'main_formation', 'year']

In [ ]:
def compare_effects(df, numerical_independent, categorical_hue, dependent_variable, fig_dimensions = (25,75)):
    '''
        INPUT:
        df: dataframe
        numerical_independent: factor which we measure against other categorical variables
        categorical_hue: categorical variablees we plot against
        dependent_variable: out main target to evaluate
        log_scale: whether you need to transform x axis into logarithmic scale or not
        OUTPUT:
        scatter plots
                                                                            '''
    fig = plt.figure(figsize = fig_dimensions)
    i = 0
    for cat in categorical_hue:
        # To show only values present in each category
        hue = df.query('{} != "nan"'.format(dependent_variable))[cat]
        i = i + 1
        ax = fig.add_subplot(len(numerical_independent), len(categorical_hue)/2, i)      
        sns.scatterplot(data = df, x = numerical_independent, y = dependent_variable, hue = hue)
        plt.suptitle('{} vs {} and {}'.format(dependent_variable, numerical_independent, cat), y = 0.89)
        if df[numerical_independent].max() - df[numerical_independent].min() > 1000:
            ax.set_xscale('log')
        

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'propp_mass_lb')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'perf_friction_psig')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'slurry_rate_bpm')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'fluid_eff_frac')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'near_wellbore_friction')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'pad_perc_as_fraction')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'net_pressure_psig')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'gel_retained_factor')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'propped_width_in')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'dimensionless_effectived_fcd')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'avg_conductivity_md_ft')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'frac_width_ft')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'frac_height_ft')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'gel_concentration_lb_mgal')

In [ ]:
for col in numerical_data:
    compare_effects(df, col, categorical_data, 'max_treating_pressure_psig')

**Conclusion**
1. For the same net pressure, Proppant size of 16/30 resulted in the highest slurry pumping rate & avg. HHP while proppant size of 12/18 required the least to achieve.
2. For the same fluid efficiency, Proppant size of 20/40 results in the highest max treating surfcace pressure while 12/18 resulted in the lowest values.
3. With other variables constant, Proppant size 12/18 resulted in the highest average conductivity.
4. With other variables constant, Conventional frac results in the highest slurry pumping rate while hiway required the least.
5. For the same frac dimensions and conductivity, Bahareya requires the lowest HHP and gel concentration, resulted in the lowest (net pressure / max treating sufrace pressure).
---------------------------------
**Comments**
* For the same pumped mass proppant, it is clear that size of 20/40 results in the highest max treating surfcace pressure.
* For the same pumping slurry rate, it is clear that size of 20/40 results in the highest max treating surfcace pressure.
* For the same fluid efficiency, it is clear that size of 20/40 results in the highest max treating surfcace pressure while 16/30 proppant size results in max treating pressure much higher than 12/18. Proppant size of 12/18 resulted in the highest average conductivity. Bahareya requirered the minimum gel concentration.
* For the samee net pressure values, it is clear that conventional frac type resulted in the highest slurry pumping rate while hiway required the least. As well, proppant size of 16/30 resulted in the highest slurry pumping rate & avg. HHP while proppant size of 12/18 required the least. proppant size of 12/18 resulted in the highest conductivity.
* For the same gel retained factor, Hiway fracture requires the least slurry pumping rate. proppant size of 12/18 results in the higheest conductivitiy.
* For the same dimensionless fcd, Bahareya requires the lowest HHP and the lowest net pressure.
* For the same frach height, Hiway required the highest gel retained factor

# Q0:
**Does success rate of different frac jobs depend on the formation?**

In [ ]:
# Conversion rate for frac jobs among different formations
conversion_rate = pd.DataFrame(df.groupby('main_formation')['success'].value_counts(0).rename('jobs'))
conversion_rate

We can readily see that the success of frac jobs in different formations vaired such that:
* Apollonia success rate is 100% (3/3)
* Bahareya success rate is 91.3% (42/46)
* ARG success rate is 90.14% (64/71)
* AR success rate is 76.92% (10/13)
* Safa success rate is 68.75% (11/16)
* Kharite success rate is 33.33% (2/6)

# Q1:
**What is the most used polymer type and size?**

In [ ]:
# Making a pre-defined function to plot countplots
def plot_countplot(df, data, columns):
    '''
        INPUTS
        df: Dataframe
        data: columns to plot
        columns: number of columns to be plotted in subplots
        OUTPUT
        countplot graphs of all the columns of input
                                                                    '''
    # Making a figure to plot data into
    fig = plt.figure(figsize = (20,10))
    # number of rows in the subplot
    rows = round(len(data)/columns)
    for i, col in enumerate(data):
        # Adding subplots of i figures
        fig.add_subplot(rows,columns, i+1)
        # using countplot of seaborn library and sorting the categories descendingly
        sns.countplot(data = df, y = col, color = 'teal', order = df[col].value_counts().index)
        # Assigning xlabels and ylabels
        plt.xlabel('Frequency')
    plt.suptitle('Frequencies for categorical features', fontsize = 16)

In [ ]:
plot_countplot(df, categorical_features, 2)

**Answer:**

1) Polymer types used:

* we can see that Ceramics is the most frequently used proppants are (Ceramics (51.6%) - Premium prop. (16.12%) - Premium plus prop. (12.9%)- ISP (4.8%) - Carbolite (4.8%)) and then come others.
* we can also see that premium plus prop. could be used with other props. in the same job based on well conditions such as (RC and Ceramics plus).


2) Polymer sizes used:

* we can seee that the most commonly used sizes are (16/30 (57.2%) - 12/80 (20.9%) - 20/40 (12.1%)) and then come other mesh sizes.
* we can also see here that we had to use various sizes in the same job based on well and operation conditions.

# Q2:
**What is the relationship between proppant type/size and the formation?**


In [ ]:
# Proppant types used with each foramtion
proppant_types = pd.DataFrame(df.groupby(['main_formation', 'success'])['proppant_type'].value_counts().rename('Jobs'))
proppant_types

In [ ]:
# Proppant sizes used with each formation
proppant_size = pd.DataFrame(df.groupby(['main_formation', 'success'])['proppant_size'].value_counts().rename('Jobs'))
proppant_size

**Answer:**
* Apollonia: Premium prop. used with differnet sizes of 12/18 and 16/30 and did not experience any failed jobs!
* AR: 33.3% of the jobs failed in such formation, and all the failed jobs were done by Ceramics although it succeeded in other jobs on same formation, as well failed proppant sizes succeeded on other occassions!
* ARG: 12.8% of the jobs failed, and we can clearly see that premium propant failed in this formation.
* Bahareya: 7.02% of the jobs faield, although the same propant type/size used in other jobs successfully.
* Kharita: 80% of the jobs failed on this formation which could require further investigation.
* Safa: 30% of the jobs failed with no reason related to prop. type or size.

# Q3:
**What is the relationship between closure pressures, ISIP and max treating pressure vs. formation?**

In [ ]:
closure_pressure_by_formation = df.groupby(['main_formation', 'success'])['closure_pressure_psig'].agg(['count', 'mean', 'min', 'max'])
closure_pressure_by_formation

In [ ]:
isip_by_formation = df.groupby(['main_formation', 'success'])['ISIP_psig'].agg(['count', 'mean', 'min', 'max'])
isip_by_formation

In [ ]:
max_treating_pressure_by_formation = df.groupby(['main_formation', 'success'])['max_treating_pressure_psig'].agg(['count', 'mean', 'min', 'max'])
max_treating_pressure_by_formation

**Answer:**

*1. Closure Pressure & ISIP*

Although avaialble colsure pressures are not so many, yet we can find some insights such that:
* For successful jobs: Apollonia has the lowest average closure pressure with 2480 psig, then comes Bahareya with closure pressure of 3845 psig, while AR and Safa have similar closure pressures around 7900 psig.
* For failed jobs: ARG and Safa have simialr closure pressures as well around 10200 psig, then comes Kharita with the highest value of 11237 psig.
We can also see that ISIP values show similar behaviour!

*2. Max treating Pressure*
* For successful jobs, we can see that Safa and Kharita have the highest values around 9440 psig, then comes AR, ARG and Apollonia with values around 5820 psig. Then Baharayea comes with the least values of 4545 psig.

# Q4:
**Indication of proppant volume and slurry volume for formations**

In [ ]:
# Slurry volume injected into each formation
slurry_vol_by_formation = df.groupby(['main_formation', 'success'])['slurry_vol_gal'].agg(['count'])
slurry_vol_by_formation

In [ ]:
# Slurry volume and designed pad volume injected against formations
df_plot_slurry_vol = df.melt(id_vars='main_formation', value_vars=["slurry_vol_gal", "pad_volume"])
plt.figure(figsize = (20,8))
sns.boxplot(x="main_formation", y="value", hue = 'variable', data = df_plot_slurry_vol);
plt.ylabel('volume in gallons')
plt.title('Designed pad volume & Slurry volume injected among various formations');

In [ ]:
# Proppant mass injected into formations 
prop_mass_by_formation = df.groupby(['main_formation', 'success'])['propp_mass_lb'].agg(['count'])
prop_mass_by_formation

In [ ]:
# Designed and actual Proppant mass injected against formations
df_plot_prop_mass = df.melt(id_vars='main_formation', value_vars=["propp_mass_lb", "propp_mass_design"])
plt.figure(figsize = (20,8))
sns.boxplot(x="main_formation", y="value", hue = "variable", data = df_plot_prop_mass);
plt.ylabel('mass in lbs')
plt.title('Designed and actual Proppant mass injected among various formations');

**Answer:**
* We can see as illustrated in the above chart that actual slurrt volume injected into formation is usually higher than the designed pad volume for various formations except for Bahareya formation which needs further investigation.
* We can also see that slighlty occurring with designed proppand mass and actual injected mass volume for Bahareya and AR fromations, while we don't have enough information about AR regarding slurry volume.
* We can clearly see that AR is the formation which receives the highest injected slurry volume, then Apollonia, Safa, Kharita and ARG. This is not true for proppant pumped into formations since here it differs such that Apollonia comes first, Safa and then comes AR, we can also see that Kharita is the least receiver for proppant mass among all formations.

# Q5:
**What is the relationship between max treating pressure, hhp, pumping rate and different formations?**

In [ ]:
# Plotting amx treating pressure against slurry pumping rate for each formation
plt.figure(figsize = (20,8))
sns.scatterplot(x="slurry_rate_bpm", y="max_treating_pressure_psig", hue = "main_formation", data = df);
plt.xticks(np.arange(5,60,5));
plt.title('Slurry inj. rate vs. Max reached treating pressure');

In [ ]:
# Setting labels orders to avoid misleading legends
labels = df['main_formation'].unique().tolist()[1:]
g = sns.FacetGrid(df, col="success", height = 8, aspect = 1)
g.map_dataframe(sns.scatterplot, x="slurry_rate_bpm", y="max_treating_pressure_psig", hue = "main_formation", hue_order = labels)
g.set_axis_labels("slurry_rate_bpm", "max_treating_pressure_psig")
g.add_legend();

In [ ]:
# Plotting amx treating pressure against slurry pumping rate for each formation
plt.figure(figsize = (20,8))
sns.scatterplot(x="slurry_rate_bpm", y="avg_hhp", hue = "main_formation", data = df);
plt.xticks(np.arange(5,60,5));
plt.title('Slurry inj. rate vs. Max reached treating pressure');

In [ ]:
# Plotting HHP against slurry pumping rate for each formation
plt.figure(figsize = (20,8))
sns.scatterplot(x="max_treating_pressure_psig", y="avg_hhp", hue = "main_formation", data = df);
plt.title('Slurry inj. rate vs. HHP');

**Answer:**

As expected from the strong correlation betweeb max treatin pressure, HHP and slurry rate, we can see positive trend in the above charts clearly!
* we can see that for the same formation, some jobs required high slurry rates while other were done using lower rates yet we can see that Safa formation usually require higher pumping rate than other formations. As well, Kharita & Apollonia usually require the lowest pumping rates. on the other hand, we can see that Bahareya, and AR usually require stable pumping rate between 20 to 40 bpm.

# Q6:
**What is the relationship between fluid efficiency and other variables and its effect on max treating pressure?**

In [ ]:
# Plotting amx treating pressure against slurry pumping rate for each formation
plt.figure(figsize = (20,8))
sns.scatterplot(x="fluid_eff", y="propp_mass_lb", size = "max_treating_pressure_psig", data = df[df['success']==1], alpha=0.5, sizes=(5, 1000));
plt.title('Fluid Eff vs. Max Treating pressure with Proppant mass injected');

In [ ]:
plt.figure(figsize = (20,8))
sns.scatterplot(x="fluid_eff", y="proppant_type", size = "max_treating_pressure_psig", data = df, alpha=0.5, sizes=(20, 600))
plt.title('Fluid Eff vs. Max Treating pressure with Proppant types');

In [ ]:
plt.figure(figsize = (20,8))
sns.scatterplot(x="fluid_eff", y="proppant_size", size = "max_treating_pressure_psig", data = df, alpha=0.5, sizes=(20, 600))
plt.title('Fluid Eff vs. Max Treating pressure with Proppant types');

In [ ]:
# Plotting amx treating pressure against slurry pumping rate for each formation
plt.figure(figsize = (20,8))
sns.scatterplot(x="fluid_eff", y="slurry_rate_bpm", data = df,);
plt.title('Fluid Eff vs. slurry inj. rate');

**Answer:**
* We can observe that as fluid efficiency increases, max treating pressure reached on surface decreases as well as saving cost by reducing amount of proppant mass injected! The highest proppant mass used are usually accompanied with low fluid efficiencies.
* we can see that some proppant types such as ISP usually accomplish low fluid efficiency and low treating pressure, while HSP and Ceramics usually require higher treating pressures!
* We can also see that proppant size of 12/18 and 16/30 could achieve high fluid efficiencies with intermediate treatment pressure, while mesh size of 20/40 usually achieve low fluid efficiency and very high treating pressures!
* we can see that there is no reationship between increasing slurry injection rate and the fluid efficiency since we achieve same fluid efficincies with various inj. rates!

# Q7:
**Does propant type/size affect closure pressure? And what role does frac type have?**

In [ ]:
plt.figure(figsize = (20,8))
sns.scatterplot(x="closure_pressure_psig", y="proppant_size", hue = "main_formation", data = df[df['success']==1]);
plt.title('Proppant size vs. closure pressure and proppant type for successful jobs');

In [ ]:
plt.figure(figsize = (20,8))
sns.scatterplot(x="closure_pressure_psig", y="proppant_type", hue = "main_formation", data = df[df['success']==1]);
plt.title('Proppant size vs. closure pressure and proppant type for successful jobs');

In [ ]:
plt.figure(figsize = (20,8))
sns.scatterplot(x="frac_type", y="closure_pressure_psig", hue = 'main_formation', data = df);
plt.title('Frac type vs. closure pressure and proppant type for successful jobs');

In [ ]:
plt.figure(figsize = (20,8))
sns.scatterplot(x="frac_type", y="fluid_eff", data = df);
plt.title('Effect of frac type on fluid efficiency');

**Answer:**
* We can see that for the same formation and successful operations, changing proppant size affected the closure pressure, sch that for Bahareya; 16/20 and 16/30 sizes caused very low closure pressure, yet a size of 20/40 tripled the closure pressure!
* we can also see that this is affected by proppant type such that Carbolite and premium caused small closure values.
* Yet we can see that this case of high closure pressure is a case of foam injection! yet we can see from the above chart that foam is not necessarily to cause increase in closure pressure, so it could be characteristics of the formatiom/prop type, size!
* we can therafter see that fluid efficiency of hiway frac is usually very low compared to conventional frac!